Pagination

The provided Python script utilizes web scraping techniques to extract information from a webpage on the Canada Revenue Agency (CRA) website. It iterates through two pages of search results, searching for organizations in Alberta ('AB') with a specific status and sorting criteria. The script uses the BeautifulSoup library to parse the webpage's HTML content, identifies relevant table rows, extracts the URLs of organizations, constructs complete URLs using a base URL, and appends them to a list called org_link_list. Ultimately, this script serves as a tool for gathering links to organizations that match the specified criteria, though the exact purpose or use of these links would require additional context

In [10]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re


org_link_list=[]
baseurl='https://apps.cra-arc.gc.ca'

for i in range(1,3):
    url = f'https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dtldAdvncdSrch?dsrdPg={i}&q.stts=0007&q.prvncSttCd=AB&q.ordrClmn=NAME&q.ordrRnk=ASC'
    print(url)
    r= requests.get(url)
    soup= bs(r.text,'html.parser')
    table = soup.find('table')  
    for row in table.find_all('tr')[1:]:
        link = row.find('a')
        if link:
            href = link.get('href')
            org_link = baseurl + href
            #print(org_link)
            org_link_list.append(org_link)
            
print(len(org_link_list))

https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dtldAdvncdSrch?dsrdPg=1&q.stts=0007&q.prvncSttCd=AB&q.ordrClmn=NAME&q.ordrRnk=ASC
https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dtldAdvncdSrch?dsrdPg=2&q.stts=0007&q.prvncSttCd=AB&q.ordrClmn=NAME&q.ordrRnk=ASC
50


In each non-profit organisation page in the CRA website, following code will extract the link for 2021 and 2022 data
This will be looped throup each link in "org_link_list".

In [12]:

url = 'https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dsplyRprtngPrd?selectedCharityBn=886474477RR0003&dsrdPg=1&q.stts=0007&q.prvncSttCd=AB'
r = requests.get(url)
soup = bs(r.text, 'html.parser')

sections = soup.find_all('section', class_='col-sm-3')

second_section = sections[1]
ul = second_section.find('ul', class_='list-unstyled')

link2022 = ul.find_all('a')[0].get('href')
print('Link for 2022 data:',baseurl + link2022)

link2021 = ul.find_all('a')[1].get('href')
print('Link for 2021 data:',baseurl + link2021)

Link for 2022 data: https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dsplyT3010FrFlngPrd?q.stts=0007&q.prvncSttCd=AB&selectedFilingPeriodIndex=0&selectedCharityBn=886474477RR0003&isSingleResult=false
Link for 2021 data: https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dsplyT3010FrFlngPrd?q.stts=0007&q.prvncSttCd=AB&selectedFilingPeriodIndex=1&selectedCharityBn=886474477RR0003&isSingleResult=false


Following code explain the senario explain how navigate through two type of non-profit organisations which are annual revenue 'less than 100k', and '100k and above'.

In [17]:


#example link for annual revenue is above 100k
url_6='https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dsplyT3010FrFlngPrd?q.stts=0007&q.prvncSttCd=AB&selectedFilingPeriodIndex=0&selectedCharityBn=108035544RR0001&isSingleResult=false'

#example link for annual revenue is less than 100k
url_d='https://apps.cra-arc.gc.ca/ebci/hacc/srch/pub/dsplyT3010FrFlngPrd?q.stts=0007&q.prvncSttCd=AB&selectedFilingPeriodIndex=0&selectedCharityBn=886474477RR0003&isSingleResult=false'

r = requests.get(url_d)
soup= bs(r.text, 'html.parser')

table = soup.find_all('div', class_= 'row')
table1= table[1]

rows= table1.find_all('div', class_='row mrgn-lft-sm')[5]
status= rows.find('div',class_='text-bold mrgn-bttm-md').text.strip()


if status == 'One or more of the above applies to the charity.  We will complete Schedule 6':
    print('look for section 06')
    # Find the link associated with "Schedule 6 - Detailed financial information"
    link1 = table1.find('a', text='Schedule 6 - Detailed financial information')

    if link1:
        # Extract the href attribute
        href = link1.get('href')
        sec_6_link= baseurl + href
        print(sec_6_link)
        r = requests.get(sec_6_link)
        soup = BeautifulSoup(r.text, 'html.parser')
        #print(soup)
    
else:
    print('look for section D ')
    status= rows.find_all('div',class_='row')[1]
    q4020 = status.find_all('div', class_='container-fluid')[0].find('div', class_='text-bold mrgn-bttm-md').text.strip()
    q4050= status.find_all('div', class_='container-fluid')[1].find('div', class_='text-bold mrgn-bttm-md').text.strip()
    q4200= status.find_all('div', class_='container-fluid')[2].find('div', class_='text-bold mrgn-bttm-md').text.strip()
    print('4020:', q4020)
    print('4050:',q4050)
    print('4200:',q4200)

look for section D 
4020: Accrual
4050: No
4200: CAN$ 28,274.00
